# Attention mechanism 

- Seq2Seq 모델의 문제점
    - Seq2Seq 모델은 Encoder에서 입력 시퀀스에 대한 특성을 **하나의 고정된 context vector**에 압축하여 Decoder로 전달 한다. Decoder는 이 context vector를 이용해서 출력 시퀀스를 만든다.
    - 하나의 고정된 크기의 vector에 모든 입력 시퀀스의 정보를 넣다보니 정보 손실이 발생한다.
    - Decoder에서 출력 시퀀스를 생성할 때 동일한 context vector를 기반으로 한다. 그러나 각 생성 토큰마다 입력 시퀀스에서 참조해야 할 중요도가 다를 수 있다. seq2seq는 encoder의 마지막 hidden state를 context로 받은 뒤 그것을 이용해 모든 출력 단어들을 생성하므로 그 중요도에 대한 반영이 안된다.

## Attention Mechanism 아이디어
-  Decoder에서 출력 단어를 예측하는 매 시점(time step)마다, Encoder의 입력 문장(context vector)을 다시 참고 하자는 것. 이때 전체 입력 문장의 단어들을 동일한 비율로 참고하는 것이 아니라, Decoder가 해당 시점(time step)에서 예측해야할 단어와 연관이 있는 입력 부분을 좀 더 집중(attention)해서 참고 할 수 있도록 하자는 것이 기본 아이디어이다.
- 다양한 Attention 종류들이 있다.
    -  Decoder에서 출력 단어를 예측하는 매 시점(time step)마다 Encoder의 입력 문장의 어느 부분에 더 집중(attention) 할지를 계산하는 방식에 따라 다양한 attention 기법이 있다.
    -  `dot attention - Luong`, `scaled dot attention - Vaswani`, `general  attention - Luong`, `concat  attention - Bahdanau` 등이 있다.

# Data Loading

In [1]:
import os
import pandas as pd
import requests

os.makedirs("data", exist_ok=True)
url = "https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv"
res = requests.get(url)
if res.status_code == 200:
    with open("data/chatbot_data.csv", "wt", encoding="utf-8") as fw:
        fw.write(res.text)
else:
    print(f"불러오지 못함: {url}")

In [2]:
df = pd.read_csv('data/chatbot_data.csv')
df.drop(columns='label', inplace=True)
df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


# 토큰화

In [4]:
question_texts = df['Q']
answer_texts = df['A']
all_texts = list(question_texts + " "+answer_texts)  # Q + A: vocab 생성.
len(question_texts), len(answer_texts), len(all_texts)

(11823, 11823, 11823)

In [9]:
all_texts

['12시 땡! 하루가 또 가네요.',
 '1지망 학교 떨어졌어 위로해 드립니다.',
 '3박4일 놀러가고 싶다 여행은 언제나 좋죠.',
 '3박4일 정도 놀러가고 싶다 여행은 언제나 좋죠.',
 'PPL 심하네 눈살이 찌푸려지죠.',
 'SD카드 망가졌어 다시 새로 사는 게 마음 편해요.',
 'SD카드 안돼 다시 새로 사는 게 마음 편해요.',
 'SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요.',
 'SNS 시간낭비인 거 아는데 매일 하는 중 시간을 정하고 해보세요.',
 'SNS 시간낭비인데 자꾸 보게됨 시간을 정하고 해보세요.',
 'SNS보면 나만 빼고 다 행복해보여 자랑하는 자리니까요.',
 '가끔 궁금해 그 사람도 그럴 거예요.',
 '가끔 뭐하는지 궁금해 그 사람도 그럴 거예요.',
 '가끔은 혼자인게 좋다 혼자를 즐기세요.',
 '가난한 자의 설움 돈은 다시 들어올 거예요.',
 '가만 있어도 땀난다 땀을 식혀주세요.',
 '가상화폐 쫄딱 망함 어서 잊고 새출발 하세요.',
 '가스불 켜고 나갔어 빨리 집에 돌아가서 끄고 나오세요.',
 '가스불 켜놓고 나온거 같아 빨리 집에 돌아가서 끄고 나오세요.',
 '가스비 너무 많이 나왔다. 다음 달에는 더 절약해봐요.',
 '가스비 비싼데 감기 걸리겠어 따뜻하게 사세요!',
 '가스비 장난 아님 다음 달에는 더 절약해봐요.',
 '가장 확실한 건 뭘까? 가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.',
 '가족 여행 가기로 했어 온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '가족 여행 고고 온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '가족 여행 어디로 가지? 온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '가족 있어? 저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '가족관계 알려 줘 저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '가족끼리 여행간다

## Tokenizer 학습

✅ 토크나이저 객체 정의 -> 어휘 사전 생성

In [11]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 10_000
min_frequency = 5 

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    continuing_subword_prefix='##',
    special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"] 
    # [SOS]: 문장의 시작을 의미하는 토큰. [EOS]: 문장이 끝난 것을 표시.
)

tokenizer.train_from_iterator(all_texts, trainer=trainer)

In [12]:
print("총 어휘수:", tokenizer.get_vocab_size())

총 어휘수: 7042


## 저장

In [13]:
dir_path = "saved_model/chatbot_attn"
os.makedirs(dir_path, exist_ok=True)
vocab_path = os.path.join(dir_path, "chatbot_attn_bpe.json")
tokenizer.save(vocab_path)

# Dataset 생성
- 한문장 단위로 학습시킬 것이므로 DataLoader를 생성하지 않고 Dataset에서 index로 조회한 질문-답변을 학습시킨다.

In [14]:
import random
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "mps" if torch.mps.is_available() else "cpu"
device

'cpu'

### Dataset 클래스 정의

In [15]:
class ChatbotDataset(Dataset):

    """
    Attribute
        max_length
        tokenizer: Tokenizer
        vocab_size: int - Tokenizer에 등록된 총 어휘수
        SOS: int - [SOS] 문장의 시작 토큰 id
        EOS: int = [EOS] 문장의 끝 토큰 id
        question_squences: list - 모든 질문 str을 token_id_list(token sequence) 로 변환하여 저장한 list 
        answser_sequences: list - 모든 답변 str을 token_id_list(token sequence) 로 변환하여 저장한 list.
    """
    def __init__(self, question_texts, answer_texts, tokenizer, min_length=2, max_length=20):
        """
        question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
        answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1",   "답2",   ...]
        tokenizer: Tokenizer
        min_length=2: int - 최소 토큰 개수. 질문과 답변의 token수가 min_length 이상인 것만 학습한다.
        max_length=20:int 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        """
        self.min_length = min_length
        self.max_length = max_length
        self.tokenizer = tokenizer
        
        self.vocab_size = tokenizer.get_vocab_size()
        self.SOS = self.tokenizer.token_to_id('[SOS]')
        self.EOS = self.tokenizer.token_to_id('[EOS]')

        # 각각의 질문, 답변 토큰 ID들을 저장할 리스트
        self.question_sequences = []
        self.answer_sequences = []
        for q, a in zip(question_texts, answer_texts):
            q_token = self.__process_sequence(q)
            a_token = self.__process_sequence(a)
            # 질문/답변 토큰의 개수가 min_length 초과인 경우만 list에 추가.
            if len(q_token) > min_length and len(a_token) > min_length:
                self.question_sequences.append(q_token)
                self.answer_sequences.append(a_token)

    def __add_special_tokens(self, token_sequence):
        """
        질문/답변 토큰 리스트 맨 뒤에 문장의 끝을 표시하는 [EOS] 토큰 추가. 
        [EOS] Token을 붙이고 max_length 보다 토큰수가 많으면 안된다.
        Args:
            token_sequence (list[str]) - EOS 토큰을 추가할 문서 token sequence
        """
        token_id_list = token_sequence[:self.max_length-1]
        token_id_list.append(self.EOS)

        return token_id_list

    def __process_sequence(self, text): 
        """
        한 문장 string을 받아서 encoding 한 뒤 [EOS] token을 추가한 token_id 리스트(list)를 생성 해서 반환한다.
        Args:
            text (str) - token id 리스트로 변환할 대상 String.
        """
        encode = self.tokenizer.encode(text)
        token_ids = self.__add_special_tokens(encode.ids)
        return token_ids
    
    def __len__(self):
        return len(self.question_sequences)

    def __getitem__(self, index): 
        # embedding 입력 -> int64
        # unsqueeze(1) - [1, 2, 3, 4] -> [[1], [2], [3], [4]]
        q = torch.tensor(self.question_sequences[index], dtype=torch.int64).unsqueeze(1)
        a = torch.tensor(self.answer_sequences[index], dtype=torch.int64).unsqueeze(1)
        return q, a

### Dataset 객체 생성

In [7]:
MAX_LENGTH = 20
MIN_LENGTH = 2
dataset = ChatbotDataset(question_texts, answer_texts, tokenizer, MIN_LENGTH, MAX_LENGTH)
print(len(dataset))

11714


# 모델

## Encoder
- seq2seq 모델과 동일 한 구조
    - 이전 코드(seq2seq)와 비교해서 forward()에서 입력 처리는 token 하나씩 하나씩 처리한다. 

![encoder](figures/attn_encoder-network_graph.png)

In [8]:
class Encoder(nn.Module):
    
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        """
        Args:
            num_vocabs: int - 총 어휘수 
            hidden_size: int - GRU의 hidden size
            embedding_dim: int - Embedding vector의 차원수 
            num_layers: int - GRU의 layer수
        """
        super().__init__()
        self.num_vocabs = num_vocabs
        self.hidden_size = hidden_size

        # 임베딩 레이어
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)

        # GRU 생성
        self.gru = nn.GRU(
            input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers
        )

    def forward(self, x, hidden):
        """
        질문의 token한개의 토큰 id를 입력받아 hidden state를 출력
        
        Args:
            x: 한개 토큰. shape-[1]
            hidden: hidden state (이전 처리결과). shape: [1, 1, hidden_size]
        Returns
            tuple: (output, hidden) - output: [1, 1, hidden_size],  hidden: [1, 1, hidden_size]
        """
        # x shape: [batch: 1]
        embedded = self.embedding(x).unsqueeze(0) # (1: batch, embedding_dim)-> (1: batch, 1:seq_len, embedding_dim)
        out, hidden = self.gru(embedded, hidden)

        return out, hidden
    
    def init_hidden(self, device):
        """
        처음 timestep에서 입력할 hidden_state. 
        값: 0
        shape: (Bidirectional(1) x number of layers(1), batch_size: 1, hidden_size) 
        """
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Attention 적용 Decoder
![seq2seq attention outline](figures/attn_seq2seq_attention_outline.png)

- Attention은 Decoder 네트워크가 순차적으로 다음 단어를 생성하는 자기 출력의 모든 단계에서 인코더 출력 중 연관있는 부분에 **집중(attention)** 할 수 있게 한다. 
- 다양한 어텐션 기법중에 **Luong attention** 방법은 다음과 같다.
  
![attention decoder](figures/attn_decoder-network_graph.png)

### Attention Weight
- Decoder가 현재 timestep의 단어(token)을 생성할 때 Encoder의 output 들 중 어떤 단어에 좀더 집중해야 하는지 계산하기 위한 가중치값.
  
![Attention Weight](figures/attn_attention_weight.png)

### Attention Value
- Decoder에서 현재 timestep의 단어를 추출할 때 사용할 Context Vector. 
    - Encoder의 output 들에 Attention Weight를 곱한다.
    - Attention Value는 Decoder에서 단어를 생성할 때 encoder output의 어떤 단어에 더 집중하고 덜 집중할지를 가지는 값이다.

![attention value](figures/attn_attention_value.png)

### Feature Extraction
- Decoder의 embedding vector와 Attention Value 를 합쳐 RNN(GRU)의 입력을 만든다.
    - **단어를 생성하기 위해 이전 timestep에서 추론한 단어(현재 timestep의 input)** 와 **Encoder output에 attention이 적용된 값** 이 둘을 합쳐 입력한다.
    - 이 값을 Linear Layer함수+ReLU를 이용해 RNN input_size에 맞춰 준다. (어떻게 input_size에 맞출지도 학습시키기 위해 Linear Layer이용)

![rnn](figures/att_attention_combine.png)

### 단어 예측(생성)
- RNN에서 찾은 Feature를 총 단어개수의 units을 출력하는 Linear에 입력해 **다음 단어를 추론한다.**
- 추론한 단어는 다음 timestep의 입력($X_t$)으로 RNN의 hidden은 다음 timestep 의 hidden state ($h_{t-1}$) 로 입력된다.


In [11]:
class AttentionDecoder(nn.Module):

    def __init__(self, num_vocabs, hidden_size, embedding_dim, dropout_p, max_length):
        # num_vocabs: 총 어휘수
        super().__init__()
        self.num_vocabs = num_vocabs
        self.hidden_size = hidden_size
        self.max_length = max_length

        # embedding layer
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)

        # attention weight를 계산하는 Linear 
        # 이전 단어의 hidden state(prev_hidden)와 현재 단어의 embedding vector에 
        #     가중합을 계산해서 attention weight를 계산. 
        ##  in_features: hidden_size+embedding_dim
        ##  out_features: Encoder의 hidden_state의 개수(max_length)
        self.attn = nn.Linear(hidden_size+embedding_dim, max_length)

        # 가정: hidden_size-200, max_length(토큰수)-20
        # attention value = attention-weight @ encoder의 hidden state들(out)
        # shape: 1 x 20 @ 20 x 200 = 1 x 200

        # 현재 단어 embedding vector + attention value를 입력받아 가중합을 계산해서 
        ##   GRU(RNN)에 입력할 입력값을 계산.
        ##  in_features: embedding_dim + encoder의 hidden_size
        self.attn_combine = nn.Linear(embedding_dim+hidden_size, hidden_size)

        self.dropout = nn.Dropout(dropout_p)
        # GRU
        self.gru = nn.GRU(hidden_size, hidden_size)

        # 분류기 
        self.classifier = nn.Linear(hidden_size, num_vocabs)

    def forward(self, x, hidden, encoder_outputs):
        """
        Parameter
            x: 현재 timestep의 입력 토큰(단어) id
            hidden: 이전 timestep 처리결과 hidden state
            encoder_outputs: Encoder output들. 
        Return
            tupe: (output, hidden, attention_weight)
                output: 단어별 다음 단어일 확률.  shape: [vocab_size]
                hidden: hidden_state. shape: [1, 1, hidden_size]
                atttention_weight: Encoder output 중 어느 단어에 집중해야하는 지 가중치값. shape: [1, max_length]
        
        현재 timestep 입력과 이전 timestep 처리결과를 기준으로 encoder_output와 계산해서  encoder_output에서 집중(attention)해야할 attention value를 계산한다.
        attention value와 현재 timestep 입력을 기준으로 단어를 추론(생성) 한다.
        """
        embedding = self.embedding(x).unsqueeze(0) # [1:batch] -> [1:batch, 1:seq_len]
        embedding = self.dropout(embedding)

        # attention weight 계산 
        # 입력: embedding vector + prev_hidden-state (합치기)
        # pytorch에서 tensor를 합치는 함수: torch.concat([합칠 대상, ..], dim=방향축)
        attn_in = torch.concat((embedding[0], hidden[0]), dim=1)
        # attn_in shape: [1:batch, embedding_dim+hidden_size]
        attn_score = self.attn(attn_in) #logit
        # attn_score shape:  1 x embedding_dim+hidden_size @ embedding_dim+hidden_size x max_length
        ####    1 x max_length 
        attn_weight = nn.Softmax(dim=-1)(attn_score)

        # attention value계산(attn_applied) - attn_weight @ encoder_hiddenstate들
        ## 1 x max_length  @ max_length x hidden_size
        # torch.bmm() - batch-wise matrix multiplication(배치단위 행렬곱)
        ## 3차원 배열을 받아서 1, 2축 기준으로 행렬곱 계산.
        ### (5, 2, 3) @ (5, 3, 5) => 2 x 3 @ 3 x 5  5개를 행렬곱 => (5, 2, 5)
        attn_value = torch.bmm(
            attn_weight.unsqueeze(0),     # (1, 1, max_length)
            encoder_outputs.unsqueeze(0), # (1, max_length, hidden_size)
        ) 
        # attn_value 결과: (1:batch, 1:seq_len , hidden_size)

        # attn_combine: gru의 input값을 생성
        ##  attn_value + embedding(concat) => Linear => ReLU
        attn_combine_in = torch.concat([
            attn_value[0], embedding[0]
        ], dim=1)
        gru_in = self.attn_combine(attn_combine_in) # 출력 (1, hidden_size)
        gru_in = gru_in.unsqueeze(0)
        gru_in = nn.ReLU()(gru_in)

        # gru에 입력해서 다음 단어를 찾기 위한 hidden state(feature)를 계산.
        out, hidden_state = self.gru(gru_in, hidden)

        # classification 에 out을 입력해서 다음 단어를 예측
        last_out = self.classifier(out[0])
        # last_out shape: [1, num_vocabs]

        return last_out[0], hidden_state, attn_weight


In [30]:
# Encoder/Decoder를 dummy_data로 확인
dummy_encoder = Encoder(
    num_vocabs=tokenizer.get_vocab_size(),
    hidden_size=256,
    embedding_dim=200,
    num_layers=1
)
dummy_encoder = dummy_encoder.to(device)

dummy_decoder = AttentionDecoder(
    num_vocabs=tokenizer.get_vocab_size(),
    hidden_size=256,
    embedding_dim=200,
    dropout_p=0.3,
    max_length=20
)
dummy_decoder = dummy_decoder.to(device)

In [31]:
x, y = dataset[0]  # 첫번째 (Q, A)
x, y = x.to(device), y.to(device)


# 첫번째 질문의 첫번째 토큰을 입력. x[0]
# hidden state (이전 처리 결과가 없으므로 0)
encoder_out, encoder_hidden = dummy_encoder(x[0], dummy_encoder.init_hidden(device))
encoder_out.shape, encoder_hidden.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [32]:
# 첫번째 답변의 첫번째 토큰을 입력. y[0]
encoder_outputs = torch.randn(20, 256, device=device) # 20: seq_len, 256: hidden_size
next_token, hidden_state, attn_weight = dummy_decoder(y[0], encoder_out, encoder_outputs)

In [36]:
print(next_token.shape)
print(next_token.argmax(-1), tokenizer.id_to_token(next_token.argmax(-1).item()))
print(hidden_state.shape)

torch.Size([7042])
tensor(6370) 길게
torch.Size([1, 1, 256])


In [38]:
print(attn_weight.shape)
attn_weight

torch.Size([1, 20])


tensor([[0.0201, 0.1395, 0.0386, 0.0760, 0.0238, 0.0669, 0.0497, 0.0437, 0.0317,
         0.0189, 0.0812, 0.0387, 0.1263, 0.0254, 0.0210, 0.0652, 0.0214, 0.0409,
         0.0202, 0.0507]], grad_fn=<SoftmaxBackward0>)

# Training

In [42]:
SOS_TOKEN = dataset.tokenizer.token_to_id("[SOS]")
EOS_TOKEN = dataset.tokenizer.token_to_id("[EOS]")
print(SOS_TOKEN, EOS_TOKEN)

2 3


In [ ]:
# 한개 question-answer 쌍을 학습
def train(
        input_tensor,  # 질문 1개
        target_tensor, # 답변 1개개
        encoder,       # Encoder
        decoder,       # AttentionDecoder
        encoder_optimizer, # encoder optimizer
        decoder_optimizer, # decoder optimizer
        loss_fn,           # loss 함수
        device, 
        max_length,
        teacher_forcing_ratio=0.9):

    input_tensor = input_tensor.to(device)
    target_tensor = target_tensor.to(target_tensor)
    loss = 0.0 # loss값 저장할 변수.
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Encoder 처리
    encoder_hidden = encoder.init_hidden(device) # 첫번째 timestep에 입력할 hidden state(0)

    # 질문/답변의 length(토큰수)를 조회
    input_length = input_tensor.shape[0]
    output_length = target_tensor.shape[0]

    # encoder hidden state들을 저장할 tensor를 정의
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    ##### 질문 문장의 (토큰별) hidden state들을 계산 -> encoder_outputs에 저장.
    for e_idx in range(input_length):
        encoder_out, encoder_hidden = encoder(input_tensor[e_idx], encoder_hidden)
        encoder_outputs[e_idx] = encoder_out

    ##### Decoder 처리(답변생성)
    # 첫번째 timestep 의 토큰: [SOS]
    decoder_input = torch.tensor([SOS_TOKEN], device=device) # decode_input: 현재 timestep의 input
    decoder_hidden = encoder_hidden # 첫번째 hidden: context vector(encoder의 마지막 hidden state)
    # teacher_forcing 여부
    teacher_forcing = True if teacher_forcing_ratio > random.random() else False

    # Decoder작업 -> 다음 단어 예측(생성)
    for d_idx in range(output_length): # 정답 토큰개수만큼 생성.
        # decoder_out: 다음 단어 예측값, decoder_hidden: GRU의 hidden state
        decoder_out, decoder_hidden, attn_weight = decoder(decoder_input, 
                                                           decoder_hidden, 
                                                           encoder_outputs)
        # loss 계산
        loss += loss_fn(decoder_out.unsqueeze(0), target_tensor[d_idx])

        # 다음 timestep에 넣을 input 토큰을 생성 -> decoder_input
        ## 정답 토큰 if teacher_forcing else Decoder예측단어
        if teacher_forcing:
            decoder_input = target_tensor[d_idx]
        else:
            output_token = decoder_out.argmax(dim=-1).unsqueeze(0)
            decoder_input = output_token.detach() # Tensor.detach(): gradient 계산그래프에서 제외.

        teacher_forcing_ratio *= 0.99
        
        if decoder_input == EOS_TOKEN: # 생성한 단어가 [EOS](문장의 끝)이면 생성 종료
            break
    
    # 순전파가 완료 (질문 -> 답변) ==> 역전파 gradient 계산-파라미터 업데이트
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    # loss 평균 반환
    return loss.item() / output_length



In [55]:
def train_iterations(
        encoder, decoder, n_iters, 
        dataset, device, log_interval=1000, learning_rate=0.001):
        # n_iters: 학습시킬 데이터(Q-A쌍)의 개수
        # log_interval: train loss를 몇개 데이터 학습마다 출력할지.

        # encoder/decoder 모델을 train 모드로 변환
        encoder.train()
        decoder.train()
        print_loss = 0.0 # 출력할 loss값 (출력하면 0으로 초기화)

        # 옵티마이저 생성
        encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
        #loss 함수
        loss_fn = nn.CrossEntropyLoss()

        # 학습 시킬 데이터를 sampling
        data_length = len(dataset)
        train_data = [dataset[random.randint(0, data_length-1)]   for i in range(n_iters)]

        # 학습-train
        s = time.time()
        for idx in range(n_iters):
            input_tensor, target_tensor = train_data[idx]
            loss = train(input_tensor, target_tensor, encoder, decoder,
                         encoder_optimizer, decoder_optimizer, loss_fn,
                         device, max_length=MAX_LENGTH
                        )
            print_loss += loss 
            if (idx+1) % log_interval == 0: 
                print(f"{idx+1}개 QA상 학습: loss - {print_loss/log_interval:.5f}")
                print_loss = 0.0

        e = time.time()
        print(f'학습에 걸린 시간: {e-s}초')
        

In [56]:
NUM_VOCABS = tokenizer.get_vocab_size()
HIDDEN_SIZE = 200
EMBEDDING_DIM = 256
DROPOUT_P = 0.2
MAX_LENGTH = 20

encoder = Encoder(NUM_VOCABS, HIDDEN_SIZE, EMBEDDING_DIM, 1)
decoder = AttentionDecoder(NUM_VOCABS, HIDDEN_SIZE, EMBEDDING_DIM, DROPOUT_P, MAX_LENGTH)

encoder = encoder.to(device)
decoder = decoder.to(device)

In [ ]:
n_iters = 100_000
log_interval = 500
train_iterations(encoder, decoder, n_iters, dataset, device, log_interval)

500개 QA상 학습: loss - 5.93458
1000개 QA상 학습: loss - 5.79095
학습에 걸린 시간: 66.70495510101318초


## Model 생성, 학습

In [ ]:
# # 하이퍼파리터들 정의
# VOCAB_SIZE = tokenizer.get_vocab_size()
# HIDDEN_SIZE = 200
# EMBEDDING_DIM = 256
# DROPOUT_P = 0.2

# # 인코더
# encoder = Encoder(VOCAB_SIZE, hidden_size=HIDDEN_SIZE, 
#                   embedding_dim=EMBEDDING_DIM, num_layers=1)

# decoder = AttentionDecoder(VOCAB_SIZE, hidden_size=HIDDEN_SIZE, embedding_dim=EMBEDDING_DIM,
#                            dropout_p=DROPOUT_P, max_length=MAX_LENGTH)
# encoder = encoder.to(device)
# decoder = decoder.to(device)

In [ ]:
# n_iters = 100000    
# log_interval = 1000

# train_iterations(encoder, decoder, n_iters=n_iters, dataset=dataset, device=device, log_interval=log_interval)

## 저장

In [58]:
# 토크나이저, 인코더, 디코드
root_path = "saved_model/chatbot_attn"
os.makedirs(root_path,  exist_ok=True)

tokenizer_path = os.path.join(root_path, "tokenizer.json")
encoder_path = os.path.join(root_path, "encoder_model.pt")
decoder_path = os.path.join(root_path, "decoder_model.pt")

tokenizer.save(tokenizer_path)
torch.save(encoder, encoder_path)
torch.save(decoder, decoder_path)

## 검증

In [59]:
root_path = "saved_model/chatbot_attn"
os.makedirs(root_path,  exist_ok=True)

tokenizer_path = os.path.join(root_path, "tokenizer.json")
encoder_path = os.path.join(root_path, "encoder_model.pt")
decoder_path = os.path.join(root_path, "decoder_model.pt")

In [60]:
# 저장된 모델 Load
tokenizer = Tokenizer.from_file(tokenizer_path)
encoder = torch.load(encoder_path, weights_only=False, map_location=device)
decoder = torch.load(decoder_path, weights_only=False, map_location=device)

In [64]:
SOS_TOKEN = tokenizer.token_to_id('[SOS]')
EOS_TOKEN = tokenizer.token_to_id('[EOS]')
def evaluate(encoder, decoder, input_tensor, dataset, device, max_length):
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        input_length = input_tensor.shape[0]  # 질문 문장 토큰 개수.
        encoder_hidden = encoder.init_hidden(device) # 첫 timestep에 넣어줄 hidden state

        # encoder의 hidden state들을 모을 텐서 생성
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        # encoder 실행
        for e_index in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[e_index], encoder_hidden)
            encoder_outputs[e_index] = encoder_output[0, 0]

        # decoder 실행
        decoder_input = torch.tensor([SOS_TOKEN], device=device)
        decoder_hidden = encoder_hidden

        # 결과를 저장할 리스트
        decoded_words = []  # 디코더가 추론한 단어(토큰)들을 저장.
        decoder_attn_weights = [] # 각 단어들을 추론할 때 계산된 attention weight값들을 저장.

        for d_index in range(max_length):
            decoder_output, decoder_hidden, attn_weight = decoder(decoder_input, 
                                                                  decoder_hidden, 
                                                                  encoder_outputs)
            decoder_attn_weights.append(attn_weight.data)

            topv, topi  = decoder_output.data.topk(1)

            if topi.item() == EOS_TOKEN:
                decoded_words.append('[EOS]')
                break
            else:
                decoded_words.append(dataset.tokenizer.id_to_token(topi.item()))
            decoder_input = topi.detach()

    return decoded_words, decoder_attn_weights

In [65]:
def handle_special_tokens(decoded_string):
    """
    Subword 처리
    subword는 단어의 시작으로 쓰인 것과 중간 부분(연결)에 사용된 두가지 subword가 있다.  연결 subword는 `#`과 같은 특수문자로 시작 한다.
    tokenizer.decode() 결과 문자열은 subword의 특수문자('##')을 처리하지 않는다. 이것을 처리하는 함수
    ex) "이 기회 ##는 내 ##꺼 #야" ==> "이 기회는 내꺼야"
    
    Parameter
        decoded_string: str - Tokenizer가 decode한 중간 subword의 특수문자 처리가 안된 문자열. 
    Return
        str: subword 특수문자 처리한 문자열
    """
    
    tokens = decoded_string.split()
    new_tokens = []
    for token in tokens:
        if token.startswith("##"):
            if new_tokens: # len(new_tokens) != 0 원소가 하나라도 있으면
                # 토큰에서 ##을 제거하고 리스트의 마지막 원소(문자열) 뒤에 붙인다.
                new_tokens[-1] += token[2:]
            else: # new_tokens가 빈 리스트. 현재 token이 첫번째 단어. ##을 지우고 append
                new_tokens.append(token[2:])
        else: # 단어의 시작인 토큰. (##이 없는 토큰) -> list에 추가.
            new_tokens.append(token)
        
    return " ".join(new_tokens) 

In [12]:
def evaluate_randomly(encoder, decoder, dataset, device, n=10):
    # n개 확인.
    for i in range(n):
        idx = random.randint(0, len(dataset))
        x, y = dataset[idx]
        q = dataset.tokenizer.decode(x.flatten().tolist())
        a = dataset.tokenizer.decode(y.flatten().tolist())
        print("질문(정답):", handle_special_tokens(q))
        print("답변(정답):", handle_special_tokens(a))

        # 추론
        output_words, atten_weights = evaluate(encoder, decoder,
                                              x.to(device), 
                                              dataset, device, MAX_LENGTH)
        # output_words: [단어, 단어, 단어, ....]
        output_sentence = ' '.join(output_words[:-1]) # [EOS]는 제거
        print("답변(예측):", handle_special_tokens(output_sentence))
        print("="*50)

In [71]:
evaluate_randomly(encoder, decoder, dataset, device, n=10)

질문(정답): 이 세상에서 내가 제일 불행해 .
답변(정답): 그럴리가요 .
답변(예측): 다시 무슨 .
질문(정답): 치매가 더 심해지는 것 같아 무서워
답변(정답): 더 많이 연락하고 관심을 가져주세요 .
답변(예측): 그런 생각 .
질문(정답): 좋아하는 사람 앞에선 다 똑같은가봐요 .
답변(정답): 누구나 좋아하는 사람 앞에서 그렇대요 .
답변(예측): 이제 더 해보세요 .
질문(정답): 짝남이 자꾸 꿈에 나오는데 꿈에서도 용기 없어서 피하고 그래 .
답변(정답): 은연 중에 지금의 심리 상태가 반영되었나봐요 .
답변(예측): 그런 무슨 .
질문(정답): 남동생한테 자꾸 화내게 되네
답변(정답): 화를 참는 연습을 해보세요 .
답변(예측): 그런 .
질문(정답): 힘든 밤입니다
답변(정답): 힘내세요 .
답변(예측): 이제 더 .
질문(정답): 잊기가 힘듭니다 .
답변(정답): 충분히 힘들만하다고 생각해요 .
답변(예측): 무슨 .
질문(정답): 오늘 기분은 설명이 안돼
답변(정답): 좋았으면 좋겠네요 .
답변(예측): 그런 .
질문(정답): 여자친구 만나고왔어 ~
답변(정답): 좋은 만남이었길 바랍니다 .
답변(예측): 그런 .
질문(정답): 이별 통보 받은지 7일째
답변(정답): 자신의 마음에 귀를 기울여보세요 .
답변(예측): 그런 .
